In [ ]:
from aiida.manage import load_profile

load_profile()

In [ ]:
from aiida.engine import submit
from aiida.orm import Dict, load_code, load_group, load_node
from aiida.plugins.factories import DataFactory

In [ ]:
code = load_code(571)
builder = code.get_builder()

structure = load_node(106)
builder.structure = structure

pseudo_family = load_group('SSSP/1.2/PBE/efficiency')
pseudos = pseudo_family.get_pseudos(structure=structure)
builder.pseudos = pseudos

parameters = {
    'CONTROL': {
        'calculation': 'scf',  # self-consistent field
    },
    'SYSTEM': {
        'ecutwfc': 30.,  # wave function cutoff in Ry
        'ecutrho': 240.,  # density cutoff in Ry
    },
}
builder.parameters = Dict(parameters)

KpointsData = DataFactory('core.array.kpoints')
kpoints = KpointsData()
kpoints.set_kpoints_mesh([4, 4, 4])
builder.kpoints = kpoints

builder.metadata.options.resources = {'num_machines': 1}

calcjob_node = submit(builder)